In [2]:
from ast import *
def _sub_subtrees_for_vars(ast, ast_mappings):
    for node in walk(ast):
        if isinstance(node, Name) and ast_mappings.has_key(unparse(ast)):
            node = ast_mappings[unparse(ast)]
    return ast
def strip_parse(expr):
    return parse(expr)
def sub_for_func(expr, func_name, func_vars, func_expr):
    ast = strip_parse(expr)
    func_name_ast = strip_parse(func_name)
    nodes = [node for node in walk(func_name_ast.body[0])]
    print(nodes)
    if not isinstance(nodes[1], Name):
        raise ValueError('Function name is not a simple name.')
    
    func_name = func_name_ast.body[0].value.id
    print(func_name)
    func_expr_ast = strip_parse(func_expr)
    # We can strip_parse  the '*', so we special case it here.
    if func_vars == '*':
        if not hasattr(func_expr_ast, 'nodes'):
            raise ValueError("Top-level function in %s does not appear to "
                             "accept variable number of arguments. (It has no "
                             "'nodes' attribute.)" % func_expr)

        func_var_names = '*'
    else:
        func_vars_ast = [strip_parse(var).body[0].value for var in func_vars]
        for var_ast in func_vars_ast:
            if not isinstance(var_ast, Name):
                raise ValueError('Function variable is not a simple name.')
        func_var_names = [getattr(var_ast, 'id') for var_ast in func_vars_ast]
        print(func_var_names)

    ast = _sub_for_func_ast(ast, func_name, func_var_names, func_expr_ast)
#     simple = Simplify._simplify_ast(ast)
    return ast


def _sub_for_func_ast(ast, func_name, func_vars, func_expr_ast):

    print(dump(ast), func_name, func_vars, dump(func_expr_ast))
    for node in walk(ast):
    
        if isinstance(node, Call) and unparse(node.value.id) == func_name\
           and func_vars == '*':
            working_ast = copy.deepcopy(func_expr_ast)
            new_args = [_sub_for_func_ast(arg_ast, func_name, func_vars, 
                                          func_expr_ast) for arg_ast in node.value.args]
            # This subs out the arguments of the original function.
            working_ast.node.value.args = new_args
            return working_ast
        if isinstance(ast, Call) and unparse(node.value.id) == func_name\
           and len(node.value.args) == len(func_vars):
            # If our ast is the function we're looking for, we take the ast
            #  for the function expression, substitute for its arguments, and
            #  return
            working_ast = copy.deepcopy(func_expr_ast)
            mapping = {}
            for var_name, arg_ast in zip(func_vars, node.value.args):
                subbed_arg_ast = _sub_for_func_ast(arg_ast, func_name, func_vars, 
                                                   func_expr_ast)
                mapping[var_name] = subbed_arg_ast
            _sub_subtrees_for_vars(working_ast, mapping)
            return working_ast
    return ast
    

In [3]:
def test_sub_for_func():
    cases = [('f(x)', 'f', 'y', 'y+1',
              'x+1'),
             ]

    for expr, func_name, func_vars, func_expr, answer in cases:
        print
        subbed = sub_for_func(expr, func_name, func_vars,
                                           func_expr)
        assert eval(answer) == eval(subbed)

In [4]:
test_sub_for_func()

[<ast.Expr object at 0x7fc70d858490>, <ast.Name object at 0x7fc70d858040>, <ast.Load object at 0x7fc7113ccb50>]
f
['y']
Module(body=[Expr(value=Call(func=Name(id='f', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]))], type_ignores=[]) f ['y'] Module(body=[Expr(value=BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Constant(value=1)))], type_ignores=[])


AttributeError: 'Call' object has no attribute 'value'

In [5]:
import ast
flags = ast.PyCF_ONLY_AST 
tree = compile("a+b", filename='<unknown>', mode='exec',
    flags=flags, dont_inherit=True)

In [1]:
import sys
sys.executable

'/usr/bin/python3.9'

In [52]:
def test__collect_num_denom():
        cases = [
                 (strip_parse('1 + 3/2 + 4'), (['1'], ['2'])),
                 ]
        for ast, (nums, denoms) in cases: 
            n, d = [], []
            print(dump(ast))
            _collect_num_denom(ast, n, d)
            n = [unparse(term) for term in n]
            d = [unparse(term) for term in d]
#             print("numerator", nums)
#             print("denominator", denoms)
#             print(n)
#             print(d)
#             assert set(nums) == set(n)
#             assert set(denoms) == set(d)

In [56]:
test__collect_num_denom()

Module(body=[Expr(value=BinOp(left=BinOp(left=Constant(value=1), op=Add(), right=BinOp(left=Constant(value=3), op=Div(), right=Constant(value=2))), op=Add(), right=Constant(value=4)))], type_ignores=[])
numerator ['1']
denominator ['2']
[]
[]


In [60]:
class PowForDoubleStar(NodeVisitor):
    nums = nums
    denoms = denoms
    def visit_BinOp(self, node):
        node.left = self.visit(node.left)
        node.right = self.visit(node.right)
        print(node)
        print(node)
        if not (isinstance(node.op, Mult) or isinstance(node.op, Div)):
            self.nums.append(node)
           
        if isinstance(node.left, Div) or isinstance(node.left, Mult):
            _collect_num_denom(node.left, nums, denoms)
        else:
            self.nums.append(node.left)
        if isinstance(node.right, Div) or isinstance(node.right, Mult):
            if isinstance(node, Mult):
                _collect_num_denom(node.right, nums, denoms)
            elif isinstance(node, Div):
                _collect_num_denom(node.right, denoms, nums)
        else:
            if isinstance(node, Mult):
                self.nums.append(node.right)
            elif isinstance(node, Div):
                denoms.append(node.right)
        self.generic_visit(node)
        

      
    
def collect_num_denom(tree, nums, denoms):
    """
    Append to nums and denoms, respectively, the nodes in the numerator and 
    denominator of an AST.
    """
    PowForDoubleStar().visit(tree, nums, denoms)

NameError: name 'nums' is not defined

In [154]:
x = "1/2"
def strip_parse(x):
    return parse(x).body[0].value
nums = []
denoms = []
class PowForDoubleStar(NodeVisitor):
    def visit_BinOp(self, node):
        node.left = self.visit(node.left)
        node.right = self.visit(node.right)
        print(node.op)
        if not (isinstance(node.op, Mult) or isinstance(node.op, Div)):
            print("here", node)
           
#         if isinstance(node.left, Div) or isinstance(node.left, Mult):
#             print("entered here")
#         else:
#             nums.append(node.left)
#         if isinstance(node.right, Div) or isinstance(node.right, Mult):
#             if isinstance(node, Mult):
#                 _collect_num_denom(node.right, nums, denoms)
#             elif isinstance(node, Div):
#                 _collect_num_denom(node.right, denoms, nums)
#         else:
#             if isinstance(node, Mult):
#                 nums.append(node.right)
#             elif isinstance(node, Div):
#                 denoms.append(node.right)
        self.generic_visit(node)
tree = strip_parse(x)
print(dump(tree))
PowForDoubleStar().visit(tree)
print(nums)
print(denoms)

BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2))
[]
[]


In [149]:
x = "(1/2)/3"
tree = strip_parse(x)
print(dump(tree))
# print(tree.body[0].value)
nums = []
denoms = []
# print("children", tree.iter_child_nodes)
_collect_num_denom(tree.body[0].value, nums, denoms)
print("nums", nums)

print("denoms", denoms)

Module(body=[Expr(value=BinOp(left=BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2)), op=Div(), right=Constant(value=3)))], type_ignores=[])
nums [1]
denoms [2, 3]


In [144]:
x = "1/2"
tree = strip_parse(x)
print(dump(tree))
print(tree.body[0].value)
print(tree.body[0].value.left)
print(tree.body[0].value.right)

Module(body=[Expr(value=BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2)))], type_ignores=[])


In [199]:
def test__collect_num_denom():
        cases = [(strip_parse('1'), (['1'], [])),
                 (strip_parse('1/2'), (['1'], ['2'])),
                 (strip_parse('1/2*3'), (['1', '3'], ['2'])),
#                  (strip_parse('1/(2*3)'), (['1'], ['2', '3'])),
#                  (strip_parse('1/(2/3)'), (['1', '3'], ['2']))
                 ]
        for ast, (nums, denoms) in cases: 
            n, d = [], []
            print(dump(ast))
            _collect_num_denom(ast, n, d)
           
            n = [term for term in n]
            d = [term for term in d]
            nums = [int(term) for term in nums]
            denoms = [int(term) for term in denoms]
           
            print(n, nums)
            print(d, denoms)
            print("-------------")
            assert set(nums) == set(n)
            assert set(denoms) == set(d)
test__collect_num_denom()

Constant(value=1)
[1] [1]
[] []
-------------
BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2))
[1] [1]
[2] [2]
-------------
BinOp(left=BinOp(left=Constant(value=1), op=Div(), right=Constant(value=2)), op=Mult(), right=Constant(value=3))


AttributeError: 'BinOp' object has no attribute 'value'

In [200]:
def _collect_num_denom(ast, nums, denoms):
    """
    Append to nums and denoms, respectively, the nodes in the numerator and 
    denominator of an AST.
    """
    if not isinstance(ast, BinOp):
        nums.append(ast.value)
        return
    if (isinstance(ast.op, Div) or isinstance(ast.op, Mult)):
        if isinstance(ast.left, BinOp):
            _collect_num_denom(ast.left, nums, denoms)
        else:
            nums.append(ast.left.value)
            print("here", nums)
            
        if isinstance(ast.right, BinOp):
            if isinstance(ast.op, Div):
                _collect_num_denom(ast.right, denoms, nums)
            elif isinstance(ast.op, Mult):
                _collect_num_denom(ast.left, nums, denoms)
        else:
            if isinstance(ast.op, Div):
                denoms.append(ast.right.value)
            elif isinstance(ast.op, Mult):
                nums.append(ast.right.value)

In [142]:
def ast2str(ast, outer = _FARTHEST_OUT, adjust = 0):
    """
    Return the string representation of an AST.

    outer: The AST's 'parent' node, used to determine whether or not to 
        enclose the result in parentheses. The default of _FARTHEST_OUT will
        never enclose the result in parentheses.

    adjust: A numerical value to adjust the priority of this ast for
        particular cases. For example, the denominator of a '/' needs 
        parentheses in more cases than does the numerator.
    """
    if isinstance(ast, Name):
        out = ast.name
    elif isinstance(ast, Const):
        out = str(ast.value)
    elif isinstance(ast, Add):
        out = '%s + %s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast))
    elif isinstance(ast, Sub):
        out = '%s - %s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast, adjust = TINY))
    elif isinstance(ast, Mul):
        out = '%s*%s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast))
    elif isinstance(ast, Div):
        # The adjust ensures proper parentheses for x/(y*z)
        out = '%s/%s' % (ast2str(ast.left, ast),
                           ast2str(ast.right, ast, adjust = TINY))
    elif isinstance(ast, Power):
        # The adjust ensures proper parentheses for (x**y)**z
        out = '%s**%s' % (ast2str(ast.left, ast, adjust = TINY),
                          ast2str(ast.right, ast))
    elif isinstance(ast, UnarySub):
        out = '-%s' % ast2str(ast.expr, ast)
    elif isinstance(ast, UnaryAdd):
        out = '+%s' % ast2str(ast.expr, ast)
    elif isinstance(ast, CallFunc):
        args = [ast2str(arg) for arg in ast.args]
        out = '%s(%s)' % (ast2str(ast.node), ', '.join(args))
    elif isinstance(ast, Subscript):
        subs = [ast2str(sub) for sub in ast.subs]
        out = '%s[%s]' % (ast2str(ast.expr), ', '.join(subs))
    elif isinstance(ast, Slice):
        out = '%s[%s:%s]' % (ast2str(ast.expr), ast2str(ast.lower), 
                             ast2str(ast.upper))
    elif isinstance(ast, Sliceobj):
        nodes = [ast2str(node) for node in ast.nodes]
        out = ':'.join(nodes)
    elif isinstance(ast, Compare):
        expr = ast2str(ast.expr, ast, adjust=6+TINY)
        out_l = [expr]
        for op, val in ast.ops:
            out_l.append(op)
            out_l.append(ast2str(val, ast, adjust=6+TINY))
        out = ' '.join(out_l)
    elif isinstance(ast, And):
        nodes = [ast2str(node, ast, adjust=TINY) for node in ast.nodes]
        out = ' and '.join(nodes)
    elif isinstance(ast, Or):
        nodes = [ast2str(node, ast, adjust=TINY) for node in ast.nodes]
        out = ' or '.join(nodes)
    elif isinstance(ast, Not):
        out = 'not %s' % ast2str(ast.expr, ast, adjust=TINY)

    # Ensure parentheses by checking the _OP_ORDER of the outer and inner ASTs
    if _need_parens(outer, ast, adjust):
        return out
    else:
        return '(%s)' % out

In [250]:
print(dump(strip_parse('x**-y')))
y = strip_parse('---x')
#  if abs(count) != 1:
#             ast_out = BinOp(left=Constant(value=abs(count), op=Mult(), right=Name(id='ast_out', ctx=Load()))
                                         
        # if count < 0:
        #     ast_out = UnaryOp(op=USub(), operand=Name(id='ast_out', ctx=Load()))

BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=UnaryOp(op=USub(), operand=Name(id='y', ctx=Load())))


In [249]:
y.operand.operand.operand

In [281]:
print(dump(strip_parse('x < 3 and not y > 4')))
# print(strip_parse('x'))

BoolOp(op=And(), values=[Compare(left=Name(id='x', ctx=Load()), ops=[Lt()], comparators=[Constant(value=3)]), UnaryOp(op=Not(), operand=Compare(left=Name(id='y', ctx=Load()), ops=[Gt()], comparators=[Constant(value=4)]))])


In [270]:
x = strip_parse('g(x-0+2, y**2 - 0**0, z*y + x/1)')
print(x.args)
# print(x.right)
# print(x.op)
x = [1,2]
y = []
x.count(1)-y.count(1)

[<ast.BinOp object at 0x7fc70474da90>, <ast.BinOp object at 0x7fc70474d670>, <ast.BinOp object at 0x7fc70474e850>]


1

In [275]:
x = strip_parse('x')
y = strip_parse('x')
print(x is y)
from ast import *

TINY = 1e-12

def _node_equal(self, other):
    """
    Return whether self and other represent the same expressions.
    Unfortunately, the Node class in Python 2.3 doesn't define ==, so
    we need to write our own.
    """
    # We're not equal if other isn't a Node, or if other is a different class.
    if not isinstance(other, Node) or not isinstance(other, self.__class__):
        return False
    # Loop through all children, checking whether they are equal
    for self_child, other_child in zip(self.getChildren(), other.getChildren()):
        if not self_child == other_child:
            return False
    # If we get here, our two nodes much be equal
    return True
Node = AST
Node.__eq__ = _node_equal
y.__eq__(x)

False


AttributeError: 'Name' object has no attribute 'getChildren'

In [300]:
x=strip_parse('f(x)')
print(dump(x))


Call(func=Name(id='f', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[])


In [299]:
x.id

'x'

In [304]:
x=strip_parse('g(x)')
print(dump(x))
print(x.args)

Call(func=Name(id='g', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[])


In [ ]:
def sub_for_func(expr, func_name, func_vars, func_expr):
    ast = strip_parse(expr)
    func_name_ast = strip_parse(func_name)
    if not isinstance(func_name_ast, Name):
        raise ValueError('Function name is not a simple name.')
    func_name = func_name_ast.name

    func_expr_ast = strip_parse(func_expr)
    # We can strip_parse  the '*', so we special case it here.
    if func_vars == '*':
        if not hasattr(func_expr_ast, 'nodes'):
            raise ValueError("Top-level function in %s does not appear to "
                             "accept variable number of arguments. (It has no "
                             "'nodes' attribute.)" % func_expr)

        func_var_names = '*'
    else:
        func_vars_ast = [strip_parse(var) for var in func_vars]
        for var_ast in func_vars_ast:
            if not isinstance(var_ast, Name):
                raise ValueError('Function variable is not a simple name.')
        func_var_names = [getattr(var_ast, 'name') for var_ast in func_vars_ast]

    ast = _sub_for_func_ast(ast, func_name, func_var_names, func_expr_ast)
    simple = Simplify._simplify_ast(ast)
    return ast2str(simple)


In [308]:
x=strip_parse('or_func')
print(x)
print(dump(x))
print(x.values)

Name(id='or_func', ctx=Load())


AttributeError: 'Name' object has no attribute 'values'

In [315]:
x=strip_parse('x**y')
print(dump(x))
print(x)
y = BinOp(left=Name(id='x', ctx=Load()), op=Call(Name('pow')), right=Name(id='y', ctx=Load()))
print(dump(y))

BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=Name(id='y', ctx=Load()))
BinOp(left=Name(id='x', ctx=Load()), op=Call(func=Name(id='pow')), right=Name(id='y', ctx=Load()))


In [318]:
x=strip_parse('1')
type(x.value)

int

In [326]:
x=strip_parse("a and b")
ops = [('&&', node) for node in x.values[1:]]
x = Compare(x.values[0], ops)
print(dump(x))
y=strip_parse("a and b")
print(dump(y))

Compare(left=Name(id='a', ctx=Load()), ops=[('&&', <ast.Name object at 0x7fc7041d76a0>)])
BoolOp(op=And(), values=[Name(id='a', ctx=Load()), Name(id='b', ctx=Load())])


In [328]:
x=strip_parse("not b")
dump(x)

"UnaryOp(op=Not(), operand=Name(id='b', ctx=Load()))"

In [336]:
y = strip_parse("a/b")
dump(y)
# Div((Mult((USub(num), denom_d)), Pow((denom, Constant(2)))))
num =1 
denom_d = 2
denom = 3
z = strip_parse('(-num * denom_d) / (denom ** 2)')
dump(z)

"BinOp(left=BinOp(left=UnaryOp(op=USub(), operand=Name(id='num', ctx=Load())), op=Mult(), right=Name(id='denom_d', ctx=Load())), op=Div(), right=BinOp(left=Name(id='denom', ctx=Load()), op=Pow(), right=Constant(value=2)))"

In [338]:
y = strip_parse('a+b')
dump(y)

"BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load()))"

In [346]:
x = strip_parse('f(x)')
print(dump(x))
dump(x.func)
x.args

Call(func=Name(id='f', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[])


In [345]:
x = strip_parse('x')
print(dump(x))

Name(id='x', ctx=Load())


In [348]:
x = strip_parse('x*y')
x1 = strip_parse('+x')
print("x1", dump(x1))
x2 = strip_parse('-x')
print("x2", dump(x2))
print(dump(x))

x1 UnaryOp(op=UAdd(), operand=Name(id='x', ctx=Load()))
x2 UnaryOp(op=USub(), operand=Name(id='x', ctx=Load()))
BinOp(left=Name(id='x', ctx=Load()), op=Mult(), right=Name(id='y', ctx=Load()))


In [351]:
x = strip_parse('f(x, y)')
print(dump(x))

Call(func=Name(id='f', ctx=Load()), args=[Name(id='x', ctx=Load()), Name(id='y', ctx=Load())], keywords=[])


In [ ]:
('pow', 2): ('arg1 * arg0**(arg1-1)', 'log(arg0) * arg0**arg1')

In [ ]:
BinOp(left=Name(id='arg1', ctx=Load()), op=Mult(), right=BinOp(left=Name(id='arg0', ctx=Load()), op=Pow(), right=BinOp(left=Name(id='arg1', ctx=Load()), op=Sub(), right=Constant(value=1)))) BinOp(left=Call(func=Name(id='log', ctx=Load()), args=[Name(id='arg0', ctx=Load())], keywords=[]), op=Mult(), right=BinOp(left=Name(id='arg0', ctx=Load()), op=Pow(), right=Name(id='arg1', ctx=Load())))

In [352]:
x=BinOp(left=BinOp(left=Name(id='y', ctx=Load()), op=Mult(), right=BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=BinOp(left=Name(id='y', ctx=Load()), op=Sub(), right=Constant(value=1)))), op=Mult(), right=Constant(value=1))

In [353]:
unparse(x)

'y * x ** (y - 1) * 1'

In [356]:
import functools
import operator
value = functools.reduce(operator.mul, [1] + [1])
print(value)

1


In [357]:
[1] + [1]
x=BinOp(left=BinOp(left=Name(id='y', ctx=Load()), op=Mult(),
right=BinOp(left=Name(id='x', ctx=Load()), op=Pow(), 
right=BinOp(left=Name(id='y', ctx=Load()),
op=Sub(), right=Constant(value=1)))), op=Mult(), right=Constant(value=1))

[1, 1]

In [ ]:
(y*x) power (y-1) product 1

In [380]:
x='y * x ** (y - 1) * 1'
y = strip_parse(x)
print(y.op)
print(dump(y.left))
print(dump(y.right))

BinOp(left=Name(id='y', ctx=Load()), op=Mult(), right=BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=BinOp(left=Name(id='y', ctx=Load()), op=Sub(), right=Constant(value=1))))
Constant(value=1)


In [417]:
(dump(y.left.right.op))

'Pow()'

In [364]:
q =  BinOp(left=BinOp(left=Name(id='y', ctx=Load()), op=Pow(), right=Constant(value=2)), op=Mult(), right=BinOp(left=Name(id='x', ctx=Load()), op=Pow(), right=BinOp(left=Name(id='y', ctx=Load()), op=Sub(), right=Constant(value=1))))
print(unparse(q))

y ** 2 * x ** (y - 1)


In [ ]:
g_1(x + cos(x*z) - y*x, y**z, x - z*14)*z*y**(z - 1) - g_0(x + cos(x*z) - y*x, y**z, x - z*14)*x

In [422]:
x=strip_parse("-b")
x.operand.id

'b'

In [ ]:
g_0(x + cos(x * z) - y * x, y ** z, x - z * 14) * -x + g_1(x + cos(x * z) - y * x, y ** z, x - z * 14) ** 2

In [418]:
g(x - y*x + cos(x*z), y**z, x - 14*z)

NameError: name 'g' is not defined

In [419]:
x= Call(func=Name(id='g_0', ctx=Load()), args=[BinOp(left=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Call(func=Name(id='cos', ctx=Load()), args=[BinOp(left=Name(id='x', ctx=Load()), op=Mult(), right=Name(id='z', ctx=Load()))], keywords=[])), op=Sub(), right=BinOp(left=Name(id='y', ctx=Load()), op=Mult(), right=Name(id='x', ctx=Load()))), BinOp(left=Name(id='y', ctx=Load()), op=Pow(), right=Name(id='z', ctx=Load())), BinOp(left=Name(id='x', ctx=Load()), op=Sub(), right=BinOp(left=Name(id='z', ctx=Load()), op=Mult(), right=Constant(value=14)))], keywords=[])
unparse(x)

'g_0(x + cos(x * z) - y * x, y ** z, x - z * 14)'

In [420]:
z=Call(func=Name(id='g_1', ctx=Load()), args=[BinOp(left=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Call(func=Name(id='cos', ctx=Load()), args=[BinOp(left=Name(id='x', ctx=Load()), op=Mult(), right=Name(id='z', ctx=Load()))], keywords=[])), op=Sub(), right=BinOp(left=Name(id='y', ctx=Load()), op=Mult(), right=Name(id='x', ctx=Load()))), BinOp(left=Name(id='y', ctx=Load()), op=Pow(), right=Name(id='z', ctx=Load())), BinOp(left=Name(id='x', ctx=Load()), op=Sub(), right=BinOp(left=Name(id='z', ctx=Load()), op=Mult(), right=Constant(value=14)))], keywords=[])
unparse(z)

'g_1(x + cos(x * z) - y * x, y ** z, x - z * 14)'

In [434]:
q= BinOp(left=BinOp(left=Name(id='z', ctx=Load()), op=Mult(), right=BinOp(left=Name(id='y', ctx=Load()), op=Pow(), right=BinOp(left=Name(id='z', ctx=Load()), op=Sub(), right=Constant(value=1)))), op=Mult(), right=Constant(value=1))
print(q.op)
print()

In [436]:
z=strip_parse('None * None')
print(dump(z))

BinOp(left=Constant(value=None), op=Mult(), right=Constant(value=None))


In [446]:
z=strip_parse('f(b)')
print(dump(z))
import ast

Call(func=Name(id='f', ctx=Load()), args=[Name(id='b', ctx=Load())], keywords=[])


In [443]:
from ast import *
print(Constant(value=0))

In [30]:
z=strip_parse('a > b and 1 > 2 and c').body[0].value
print(dump(z))
nodes = z.values
ops = [('&&', node) for node in nodes[1:]]
print(ops)
for q in ops:
    Compare(left=)
ast = Compare(left=nodes[0], ops=ops)
print("dddddddddd")
print(dump(ast))


BoolOp(op=And(), values=[Compare(left=Name(id='a', ctx=Load()), ops=[Gt()], comparators=[Name(id='b', ctx=Load())]), Compare(left=Constant(value=1), ops=[Gt()], comparators=[Constant(value=2)]), Name(id='c', ctx=Load())])
[('&&', <ast.Compare object at 0x7f2700492640>), ('&&', <ast.Name object at 0x7f26fcc275e0>)]
dddddddddd
Compare(left=Compare(left=Name(id='a', ctx=Load()), ops=[Gt()], comparators=[Name(id='b', ctx=Load())]), ops=[('&&', <ast.Compare object at 0x7f2700492640>), ('&&', <ast.Name object at 0x7f26fcc275e0>)])


In [8]:
q=BoolOp(op=And(), values=[Compare(left=Name(id='X0', ctx=Load()), ops=[Lt()], comparators=[Constant(value=0.8)]), Compare(left=Name(id='T', ctx=Load()), ops=[Gt()], comparators=[Constant(value=0.3)])])
# print(unparse(q))
q.values()
Compare(left=Compare(left=Name(id='X0', ctx=Load()), ops=[Lt()], comparators=[Constant(value=0.8)]), ops=[('&&', <ast.Compare object at 0x7f0ef11c21f0>)])

TypeError: 'list' object is not callable

In [101]:
x =  strip_parse('l[1:2, 3]')
z= x.body[0].value
print(dump(z))
print(z.value)
print(z.slice[0])
print(z.ctx)

Subscript(value=Name(id='l', ctx=Load()), slice=Tuple(elts=[Slice(lower=Constant(value=1), upper=Constant(value=2)), Constant(value=3)], ctx=Load()), ctx=Load())


TypeError: 'Tuple' object is not subscriptable

In [115]:
q=Tuple(elts=[Slice(lower=Constant(value=1), upper=Constant(value=2)), Constant(value=3)])
q.elts

[<ast.Slice at 0x7fb57c4904f0>, <ast.Constant at 0x7fb57c4909d0>]

In [119]:
x =  strip_parse('l[1:2]')
z= x.body[0].value
print(z.value)
print(z.slice.lower)
dump(z)

"Subscript(value=Name(id='l', ctx=Load()), slice=Slice(lower=Constant(value=1), upper=Constant(value=2)), ctx=Load())"

In [21]:
z=Subscript(value=Name(id='alg_yp', ctx=Load()), slice=Constant(value=0), ctx=Load())
z.values
print(unparse(z))

AttributeError: 'Subscript' object has no attribute 'values'

In [66]:
x =  strip_parse('a >= b')
z= x.body[0].value
# y = Compare(left=z.values[0], ops=())
dump(z)
(dump(z))
print((z.ops[0].__class__.__name__))
# y = Compare(left=z.values[0], ops='&&', comparators=z.values[1:])
q= {"<class 'ast.GtE'>": "=", "ast.NotEq": "!=", "ast.Lt": "<", "ast.Gt": ">", "ast.GtE": ">=", "ast.LtE": "<="}
if z.ops[0] in q:
    print(z.ops[0])

GtE


In [91]:
from ast import *
x = strip_parse('a and b').body[0].value
print(dump(x))
y = Call(func=Name(id='&&', ctx=Load()), args=[x.values[0], x.values[1]], keywords=[])
print(dump(y))
print(unparse(y))
# z = BoolOp(op=Name(id='&&'), values=[Name(id='a', ctx=Load()), Name(id='b', ctx=Load())])
# print(dump(z))
# unparse(z)

BoolOp(op=And(), values=[Name(id='a', ctx=Load()), Name(id='b', ctx=Load())])
Call(func=Name(id='&&', ctx=Load()), args=[Name(id='a', ctx=Load()), Name(id='b', ctx=Load())], keywords=[])
&&(a, b)


In [100]:
v = strip_parse('a<(a+b)').body[0].value
print(dump(v))
q = unparse(v)
dump(v.body[0].value)
print(q)

Compare(left=Name(id='a', ctx=Load()), ops=[Lt()], comparators=[BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load()))])


AttributeError: 'Compare' object has no attribute 'body'

In [170]:
a='(((x)**(((1)/(n)) - (1)))/(n))'
b=strip_parse(a).body[0].value
print(unparse(b))
x**(1/n - 1)/n 

x ** (1 / n - 1) / n


In [121]:
s = {1,2,3}
s.union_update([4,5,6])

AttributeError: 'set' object has no attribute 'union_update'

In [126]:
varsByCalc={'test_max': {'T_2_maximum': (None, 50)}}
for calc in varsByCalc:
    for depVar in varsByCalc[calc]:
        varsByCalc[calc][depVar] = list(varsByCalc[calc][depVar])
        sorted(varsByCalc[calc][depVar], key=lambda varsByCalc[calc][depVar]: -1 * float('inf') if varsByCalc[calc][depVar] is None else varsByCalc[calc][depVar])
#         sorted(varsByCalc[calc][depVar])

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-126-0ebec95ad7cd>, line 5)

In [142]:
S = set([1,2,3])
T = set([1,2,3])
S-T


set()

In [138]:
a = [                                                                                                                       ('v_sP', 0.8),                                                                                                          ('v_sT', 1.0),                                                                                                          ('v_mP', 0.8),                                                                                                          ('v_mT', 0.7),                                                                                                          ('K_mP', 0.2),                                                                                                          ('K_mT', 0.2),                                                                                                          ('k_sP', 0.9),                                                                                                          ('k_sT', 0.9),                                                                                                          ('v_dP', 2.0),                                                                                                          ('v_dT', 2.0),                                                                                                          ('k_1', 1.2),                                                                                                           ('k_2', 0.2),                                                                                                           ('k_3', 1.2),                                                                                                           ('k_4', 0.6),                                                                                                           ('K_IP', 1.0),                                                                                                          ('K_IT', 1.0),                                                                                                          ('K_dP', 0.2),                                                                                                          ('K_dT', 0.2),                                                                                                          ('n', 4),                                                                                                               ('K_1P', 2.0),                                                                                                          ('K_2P', 2.0),                                                                                                          ('K_3P', 2.0),                                                                                                          ('K_4P', 2.0),                                                                                                          ('K_1T', 2.0),                                                                                                          ('K_2T', 2.0),                                                                                                          ('K_3T', 2.0),                                                                                                          ('K_4T', 2.0),                                                                                                          ('k_d', 0.01),                                                                                                          ('k_dC', 0.01),                                                                                                         ('k_dN', 0.01),                                                                                                         ('V_1P', 8.0),                                                                                                          ('V_1T', 8.0),                                                                                                          ('V_2P', 1.0),                                                                                                          ('V_2T', 1.0),                                                                                                          ('V_3P', 8.0),                                                                                                          ('V_3T', 8.0),                                                                                                          ('V_4P', 1.0),                                                                                                          ('V_4T', 1.0),                                                                                                          ]       

In [139]:
b = [                                                                                                                       ('k_2', 0.2),                                                                                                           ('K_2T', 2.0),                                                                                                          ('K_IT', 1.0),                                                                                                          ('K_dT', 0.2),                                                                                                          ('K_3T', 2.0),                                                                                                          ('K_2P', 2.0),                                                                                                          ('K_IP', 1.0),                                                                                                          ('K_dP', 0.2),                                                                                                          ('k_4', 0.6),                                                                                                           ('K_mP', 0.2),                                                                                                          ('K_mT', 0.2),                                                                                                          ('V_4T', 1.0),                                                                                                          ('v_sP', 0.8),                                                                                                          ('v_mP', 0.8),                                                                                                          ('n', 4),                                                                                                               ('v_sT', 1.0),                                                                                                          ('v_mT', 0.7),                                                                                                          ('V_1T', 8.0),                                                                                                          ('V_4P', 1.0),                                                                                                          ('V_1P', 8.0),                                                                                                          ('k_1', 1.2),                                                                                                           ('v_dT', 2.0),                                                                                                          ('k_3', 1.2),                                                                                                           ('k_sT', 0.9),                                                                                                          ('v_dP', 2.0),                                                                                                          ('k_sP', 0.9),                                                                                                          ('k_dN', 0.01),                                                                                                         ('K_1T', 2.0),                                                                                                          ('K_1P', 2.0),                                                                                                          ('K_3P', 2.0),                                                                                                          ('V_3P', 8.0),                                                                                                          ('k_dC', 0.01),                                                                                                         ('K_4T', 2.0),                                                                                                          ('V_2P', 1.0),                                                                                                          ('V_3T', 8.0),                                                                                                          ('k_d', 0.01),                                                                                                          ('K_4P', 2.0),                                                                                                          ('V_2T', 1.0),                                                                                                          ] 

In [141]:
# d = set(a).intersection(set(b))
# print(d)
e = (a) - (b)
print("wwwwwwwww", e)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [159]:
c={'Keq': 2.5, 'cell': 1.0, 'k1': 0.1, 'k2': 0.15, 'T': 0.981682983153795, 'X0': 0.018317016846205035, 'X1': 3.2339110894780743, 'S1': 0.2804808523296553, 'time': 40.0}

In [160]:
d={'S1': 0.018207307666971536, 'cell': 1.0, 'k2': 0.15, 'k1': 0.1, 'T': 0.0637255768349033, 'time': 40.0, 'X0': 0.018315878570987116, 'X1': 0.9179585445941111, 'Keq': 2.5}

In [164]:
for k, i in enumerate(c.items()):
    print(k,i)

0 ('Keq', 2.5)
1 ('cell', 1.0)
2 ('k1', 0.1)
3 ('k2', 0.15)
4 ('T', 0.981682983153795)
5 ('X0', 0.018317016846205035)
6 ('X1', 3.2339110894780743)
7 ('S1', 0.2804808523296553)
8 ('time', 40.0)


In [165]:
for k, i in enumerate(d.items()):
    print(k,i)

0 ('S1', 0.018207307666971536)
1 ('cell', 1.0)
2 ('k2', 0.15)
3 ('k1', 0.1)
4 ('T', 0.0637255768349033)
5 ('time', 40.0)
6 ('X0', 0.018315878570987116)
7 ('X1', 0.9179585445941111)
8 ('Keq', 2.5)


In [169]:
a="asd"
a.upper()

'ASD'

In [172]:
import os
c_body = ["}"]
c_body = os.linesep.join(c_body)
print(c_body)

}


In [174]:
l= []
print(len(l))
5//2

0


2

In [175]:
a=[ ('basic', <SloppyCell.ReactionNetworks.Components.Compartment object at 0x7f9ce9db9df0>),                                                                                                                                                      ('basic', <SloppyCell.ReactionNetworks.Components.Compartment object at 0x7f9ce9db9df0>),                                                                            ('x', <SloppyCell.ReactionNetworks.Components.Species object at 0x7f9ce9c98040>),                                                                                    ('A', <SloppyCell.ReactionNetworks.Components.Parameter object at 0x7f9ce9c98070>),                                                                                  ('y', <SloppyCell.ReactionNetworks.Components.Species object at 0x7f9ce9c98130>),                                                                                    ('B', <SloppyCell.ReactionNetworks.Components.Parameter object at 0x7f9ce9c98160>),                                                                                  ]   

SyntaxError: invalid syntax (<ipython-input-175-1dea6c083465>, line 1)

In [177]:
a=[('x', <SloppyCell.ReactionNetworks.Components.Species object at 0x7f9ce9c98040>),('y', <SloppyCell.ReactionNetworks.Components.Species object at 0x7f9ce9c98130>),('B', <SloppyCell.ReactionNetworks.Components.Parameter object at 0x7f9ce9c98160>)]

SyntaxError: invalid syntax (<ipython-input-177-527879f799a3>, line 1)

In [180]:
a=[('x', 1), ('y', 2)]
a.has_key('x')

AttributeError: 'list' object has no attribute 'has_key'